In [21]:
# --- 0. 필요한 라이브러리들을 가져옵니다. ---
import time
import pandas as pd
import pymysql
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [22]:
# --- 1. YES24 베스트셀러 크롤링 함수 ---
def scrape_yes24_bestsellers():
    """
    Selenium으로 여러 페이지를 넘기며 YES24 베스트셀러 정보를 가져오는 함수
    """
    BASE_URL = "http://www.yes24.com/24/Category/BestSeller"
    driver = webdriver.Chrome()
    all_books = []

    print("YES24 베스트셀러 크롤링을 시작합니다...")

    try:
        # 1페이지부터 3페이지까지 순회
        for page_num in range(1, 4):
            print(f"--- {page_num}페이지 크롤링 중 ---")

            # 페이지 URL 생성 (1페이지는 기본 URL, 2페이지부터는 ?PageNumber=2 형식)
            url = f"{BASE_URL}?PageNumber={page_num}"
            driver.get(url)

            # 페이지가 로드될 때까지 최대 10초 대기
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "bestList"))
            )

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # 베스트셀러 목록 선택 (ol 태그의 bestList id 하위의 li들)
            book_list = soup.select("#bestList > ol > li")

            for book in book_list:
                # 순위, 제목, 저자, 가격 정보 추출
                rank_tag = book.select_one("p.bb-best-num")
                title_tag = book.select_one("p.bb-book-title > a")
                author_tag = book.select_one("p.bb-book-meta > a:nth-of-type(1)")
                price_tag = book.select_one("p.bb-book-price > strong")

                if rank_tag and title_tag and author_tag and price_tag:
                    rank = rank_tag.text.strip().replace('.', '') # '1.' -> '1'
                    title = title_tag.text.strip()
                    author = author_tag.text.strip()
                    price = price_tag.text.strip() + "원"

                    all_books.append({
                        'rank': rank,
                        'title': title,
                        'author': author,
                        'price': price
                    })

        print("모든 페이지 정보 추출 완료")

    except Exception as e:
        print(f"크롤링 중 오류 발생 : {e}")

    finally:
        driver.quit()
        print("브라우저를 종료합니다.")

    return all_books

In [23]:
# --- 2. 데이터베이스 저장 함수 (YES24 맞춤) ---
def save_to_mariadb(data):
    """
    크롤링한 베스트셀러 데이터를 MariaDB에 저장하는 함수
    """
    # DB 연결 설정 (!!! 본인 환경에 맞게 수정하세요 !!!)
    try:
        db = pymysql.connect(
            host="localhost",
            user="webuser",
            password="webuser",
            database="webdb",
            charset='utf8mb4'
        )

        cursor = db.cursor()

        # 테이블 생성 (테이블 이름과 컬럼을 YES24에 맞게 변경)
        create_table_query = """
        CREATE TABLE IF NOT EXISTS yes24_bestsellers (
            id INT AUTO_INCREMENT PRIMARY KEY,
            rank VARCHAR(10) NOT NULL,
            title VARCHAR(255) NOT NULL,
            author VARCHAR(255),
            price VARCHAR(50)
        )
        """
        cursor.execute(create_table_query)

        # 데이터 저장 (컬럼을 YES24에 맞게 변경)
        insert_query = """
        INSERT INTO yes24_bestsellers (rank, title, author, price)
        VALUES (%s, %s, %s, %s)
        """

        cursor.executemany(insert_query,
                           [(d["rank"], d["title"], d["author"], d["price"]) for d in data])
        db.commit()
        print(f"{len(data)}개의 도서 데이터가 MariaDB에 성공적으로 저장되었습니다.")

    except Exception as e:
        print(f"데이터베이스 작업 중 오류 발생: {e}")
        if 'db' in locals() and db.open:
            db.rollback()

    finally:
        if 'db' in locals() and db.open:
            cursor.close()
            db.close()

In [24]:
# --- 3. 프로그램의 메인 실행 함수 ---
def main():
    """
    프로그램의 전체 실행 흐름을 제어하는 함수
    """
    print("모든 프로세스를 시작합니다...")

    book_data = scrape_yes24_bestsellers()

    if book_data:
        print(f"\n총 {len(book_data)}개의 도서 데이터를 성공적으로 가져왔습니다.")
        save_to_mariadb(book_data)
    else:
        print("가져온 도서 데이터가 없습니다.")

    print("\n모든 작업 완료!")

In [25]:
# --- 4. 스크립트의 시작점 ---
if __name__ == '__main__':
    main()

모든 프로세스를 시작합니다...
YES24 베스트셀러 크롤링을 시작합니다...
--- 1페이지 크롤링 중 ---
크롤링 중 오류 발생 : Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff74938fca5+79861]
	GetHandleVerifier [0x0x7ff74938fd00+79952]
	(No symbol) [0x0x7ff74910cada]
	(No symbol) [0x0x7ff749164457]
	(No symbol) [0x0x7ff74916471c]
	(No symbol) [0x0x7ff7491b8217]
	(No symbol) [0x0x7ff74918cb1f]
	(No symbol) [0x0x7ff7491b4f8b]
	(No symbol) [0x0x7ff74918c8b3]
	(No symbol) [0x0x7ff749155272]
	(No symbol) [0x0x7ff749156043]
	GetHandleVerifier [0x0x7ff74964b9dd+2946349]
	GetHandleVerifier [0x0x7ff749645c5a+2922410]
	GetHandleVerifier [0x0x7ff7496659e7+3052855]
	GetHandleVerifier [0x0x7ff7493aaa8e+189918]
	GetHandleVerifier [0x0x7ff7493b2a2f+222591]
	GetHandleVerifier [0x0x7ff749398ac4+116244]
	GetHandleVerifier [0x0x7ff749398c79+116681]
	GetHandleVerifier [0x0x7ff74937f058+11176]
	BaseThreadInitThunk [0x0x7ffb39ca7374+20]
	RtlUserThreadStart [0x0x7ffb3a4bcc91+33]

브라우저를 종료합니다.
가져온 도서 데이터가 없습니다.

모든 작업 완료!
